##SETUP

In [44]:
import time
import pandas as pd
# Android ADB
from subprocess import check_output,Popen
# Notebook
from IPython.display import display,clear_output
from tqdm.notebook import trange, tqdm

import sys
# Start Scrcpy once per restart
if 'started_scrcpy' not in vars():
    started_scrcpy=True
    proc = Popen(['C:\Programs\Scrcpy\scrcpy'], shell=True)
    time.sleep(3) # <-- sleep for 6''
    proc.terminate() # <-- terminate the process (Scrcpy window can be closed)
# internal
sys.path.append('./src')
import bot_core
import bot_perception

import importlib
[importlib.reload(lib) for lib in [bot_core,bot_perception] ]


# Selected units in deck (order does not matter)
#sel_units= ['crystal.png','chemist.png','monkey.png','dryad.png','hunter.png']
sel_units= ['cauldron.png','chemist.png','harlequin.png','dryad.png','zealot.png']
bot_core.select_units(sel_units,show=False)

bot = bot_core.Bot()

In [7]:
bot_core.select_units(sel_units,show=True)

['alchemist.png', 'archer.png', 'banner.png', 'blade_dancer.png', 'bruser.png', 'catapult.png', 'cauldron.png', 'chemist.png', 'clock.png', 'cold_mage.png', 'corsair.png', 'crystal.png', 'cultist.png', 'cultist_off.png', 'demo.png', 'demon_hunter.png', 'dryad.png', 'empty.png', 'engineer.png', 'executioner.png', 'fire_mage.png', 'flinger.png', 'gargoyle.png', 'harlequin.png', 'hex.png', 'hunter.png', 'inquisitor.png', 'ivy.png', 'knight_statue.png', 'lightning.png', 'meteor.png', 'mime.png', 'monkey.png', 'old', 'old_icon', 'poisoner.png', 'portal_keeper.png', 'portal_mage.png', 'priest.png', 'pyro.png', 'reaper.png', 'rogue.png', 'sentry.png', 'shaman.png', 'sharpshooter.png', 'sharpstone.png', 'summoner.png', 'thunder.png', 'time_keeper.png', 'trapper.png', 'unit_rank', 'vampire.png', 'wind_arch.png', 'witch_doc.png', 'zealot.png']
Chosen:
 ['cauldron.png', 'knight_statue.png', 'harlequin.png', 'dryad.png', 'zealot.png']


## RUN BOT

In [46]:
wait=0
combat = 0
grid_df =None
watch_ad = True
train_ai = False
# Main loop
while(1):
    output = bot.battle_screen(start=False)
    if output[1]=='fighting':
        watch_ad = True 
        wait = 0
        combat+=1
        if combat>50:
            bot.restart_RR()
            combat = 0
            continue
        battle_bar=trange(8)
        for i in battle_bar:
                time.sleep(0.2)
                # Upgrade units
                bot.mana_level([5,2,1],hero_power=True)
                # Spawn units
                bot.click(450,1360)
                # Try to merge units
                grid_df,unit_series,merge_series,df_groups,info = bot.try_merge(prev_grid=grid_df)
                clear_output()
                display([f'{combat}, {i+1}/8 {output[1]}, {info}', 'Average age: '+ str(grid_df['Age'].mean().round(2)),grid_df,unit_series,df_groups])
        # Wait until late stage in combat and if consistency is ok, not stagnate save all units for ML model
        if combat==25 and 5<grid_df['Age'].mean()<50 and train_ai:
            bot_perception.add_grid_to_dataset()
    elif output[1]=='home' and watch_ad:
        for i in range(3):
            bot.watch_ads()
        watch_ad = False
    else:
        combat=0
        output = bot.battle_screen(start=True,pve=True,floor=7)
        wait+=1
        if wait>40:
            print('RESTARTING')
            bot.restart_RR(),
            
            wait=0
        print(output[1],'wait count:',wait)

['28, 2/8 fighting, Full Grid - Merging!',
 'Average age: 43.07',
    grid_pos           unit  probability  rank  rank_prob    Age
 0    [0, 0]     zealot.png        0.884     4   0.999999    5.0
 1    [0, 1]   cauldron.png        0.868     1   1.000000    6.0
 2    [0, 2]     zealot.png        0.884     4   0.999927   19.0
 3    [0, 3]   cauldron.png        0.844     1   0.999964   50.0
 4    [0, 4]   cauldron.png        0.830     1   0.999991   14.0
 5    [1, 0]    chemist.png        0.926     3   0.999999   34.0
 6    [1, 1]     zealot.png        0.885     4   0.999977    2.0
 7    [1, 2]     zealot.png        0.846     1   0.991222   52.0
 8    [1, 3]   cauldron.png        0.904     1   0.996562    7.0
 9    [1, 4]  harlequin.png        0.929     1   1.000000   37.0
 10   [2, 0]   cauldron.png        0.838     1   1.000000   76.0
 11   [2, 1]     zealot.png        0.891     3   0.962707   82.0
 12   [2, 2]    chemist.png        0.916     4   1.000000  146.0
 13   [2, 3]     zealot.

KeyboardInterrupt: 

## Quick restart

In [3]:
# Drop co-op game
bot.restart_RR(quick_disconnect=True)

In [2]:
# Restart game
bot.restart_RR()